In [2]:
using Distributed
addprocs(4)
println(workers())

@everywhere begin
    using Pkg
    Pkg.activate(String(@__DIR__) * "/..")
    Pkg.instantiate()
end

@everywhere begin
    include("../src/Conduction.jl");
    using Plots, VegaLite, LaTeXStrings, QuadGK;
end

@everywhere begin
    range = -10.:0.5:10.;

    semiconductor = Conduction.Semiconductor(
    1.38 * 10^-23, # Boltzman constant (J.K^-1)
    1.6*10^-19, # Electron's charge (C)
    10^7, # decay constant of the assumed hydrogen-like localized state wave functions (cm^-1)
    0.1 * 1.6*10^-19, # Mode effect of the phonons (J)
    2.1 * 10^18, # intrinsic semiconductor's density (cm^-3)
    2.1 * 10^18, # Doping states' density (cm^-3)
    0.1 * 1.6 * 10^-19, # Energy to a vacant target site (J)
    -2*10^5, # Field (V.cm^-1)
    10^13, # Base electron jump rate
    -10.0, # Fermi level (J)
    2.7, # Intrinsic semiconductor's gaussian width (J)
    2.7 # Doping states' gaussian width (J)
    );

    function einsteinRelation(semiconductor, U, T)
        r = Conduction.RnnVRH(semiconductor, U, T)
        x = Conduction.xf(semiconductor, r, U, T)
        t = Conduction.t(semiconductor, r, U, T)
        return Conduction.D(semiconductor, r, x, t) / Conduction.electronMobility(
            semiconductor, r, x)
    end;

    function einsteinRelation_no_t(semiconductor, U, T)
        r = Conduction.RnnVRH(semiconductor, U, T)
        x = Conduction.xf(semiconductor, r, U, T)
        return Conduction.D_no_t(semiconductor, r) / Conduction.electronMobility(
            semiconductor, r, x)
    end;

    function criteria_D(semiconductor, U, T)
        r = Conduction.RnnVRH(semiconductor, U, T)
        return semiconductor.nu * exp(-r) * Conduction.t(semiconductor, r, U, T)
    end;

    function to_benchmark()
        pmap(x -> einsteinRelation(semiconductor, x, 300.), range);
    end
end

[2, 3, 4, 5, 6, 7, 8, 9]


LoadError: [91mOn worker 2:[39m
[91m[91mArgumentError: Package VegaLite [112f6efa-9a02-5b7d-90c0-432ed331239a] is required but does not seem to be installed:[39m[39m
[91m[91m - Run `Pkg.instantiate()` to install all recorded dependencies.[39m[39m

[91m_require at .\loading.jl:999[39m
[91mrequire at .\loading.jl:928[39m
[91m#1 at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Distributed\src\Distributed.jl:78[39m
[91m#103 at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Distributed\src\process_messages.jl:290[39m
[91mrun_work_thunk at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Distributed\src\process_messages.jl:79[39m
[91mrun_work_thunk at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Distributed\src\process_messages.jl:88[39m
[91m#96 at .\task.jl:356[39m

[91m...and 7 more exception(s).[39m


In [ ]:
@benchmark to_benchmark()

In [ ]:
addprocs(4)
println(workers())

@everywhere begin
    using Pkg
    Pkg.activate(String(@__DIR__) * "/..")
    Pkg.instantiate()
    include("../src/Conduction.jl");
    using Plots, VegaLite, LaTeXStrings, QuadGK;

    range = -10.:0.5:10.;

    semiconductor = Conduction.Semiconductor(
    1.38 * 10^-23, # Boltzman constant (J.K^-1)
    1.6*10^-19, # Electron's charge (C)
    10^7, # decay constant of the assumed hydrogen-like localized state wave functions (cm^-1)
    0.1 * 1.6*10^-19, # Mode effect of the phonons (J)
    2.1 * 10^18, # intrinsic semiconductor's density (cm^-3)
    2.1 * 10^18, # Doping states' density (cm^-3)
    0.1 * 1.6 * 10^-19, # Energy to a vacant target site (J)
    -2*10^5, # Field (V.cm^-1)
    10^13, # Base electron jump rate
    -10.0, # Fermi level (J)
    2.7, # Intrinsic semiconductor's gaussian width (J)
    2.7 # Doping states' gaussian width (J)
    );

    function einsteinRelation(semiconductor, U, T)
        r = Conduction.RnnVRH(semiconductor, U, T)
        x = Conduction.xf(semiconductor, r, U, T)
        t = Conduction.t(semiconductor, r, U, T)
        return Conduction.D(semiconductor, r, x, t) / Conduction.electronMobility(
            semiconductor, r, x)
    end;

    function einsteinRelation_no_t(semiconductor, U, T)
        r = Conduction.RnnVRH(semiconductor, U, T)
        x = Conduction.xf(semiconductor, r, U, T)
        return Conduction.D_no_t(semiconductor, r) / Conduction.electronMobility(
            semiconductor, r, x)
    end;

    function criteria_D(semiconductor, U, T)
        r = Conduction.RnnVRH(semiconductor, U, T)
        return semiconductor.nu * exp(-r) * Conduction.t(semiconductor, r, U, T)
    end;

    function to_benchmark()
        pmap(x -> einsteinRelation(semiconductor, x, 300.), range);
    end
end

In [ ]:
@benchmark to_benchmark()